In [3]:
import pandas as pd
from Tree import *

def build_investment_tree(codes: list, risk_level: int, df: pd.DataFrame) -> Tree:
    tree = Tree("Universe")
    
    # 위험등급별 weight_bounds 설정
    if risk_level in [1,2]:  # 안정추구형
        bounds = {
            5: (0, 0.15),
            4: (0, 0.4),
            3: (0, 0.2),
            2: (0, 1),
            1: (0, 1)
        }
    elif risk_level == 3:  # 위험중립형
        bounds = {
            5: (0, 0.2),
            4: (0, 0.8),
            3: (0, 1),
            2: (0, 1),
            1: (0, 1)
        }
    else:  # 적극투자형 (4, 5)
        bounds = {
            5: (0, 0.2),
            4: (0, 1),
            3: (0, 1),
            2: (0, 1),
            1: (0, 1)
        }

    # 첫번째 열과 종목코드 매핑 생성
    row_code_dict = {}
    for _, row in df.iterrows():
        row_code_dict[row.iloc[0]] = str(row['종목 코드'])
    
    #print(row_code_dict)
    # 먼저 각 코드의 level 정보 수집
    code_info = {}
    for code in codes:
        row = df[df['종목 코드'].astype(str) == str(code)].iloc[0]
        code_info[code] = {
            'level': row['level'] if not pd.isna(row['level']) else 0,
            'parent': row['부모'] if not pd.isna(row['부모']) else None,
            'risk': row['투자 가능 대상 여부']
        }
    print('----------------')
    print(code_info)
    print('----------------')
    # 트리 구축
    
    
    sorted_codes = sorted(codes, key=lambda x: code_info[x]['level'], reverse=True)
    print(sorted_codes)
    for code in sorted_codes:
        bound = bounds[code_info[code]['risk']]
        if code_info[code]['level'] != 2:
            if row_code_dict[code_info[code]['parent']] != code:
                if row_code_dict[code_info[code]['parent']] not in [node.name for node in tree.get_all_nodes()]:
                    tree.insert('Universe', row_code_dict[code_info[code]['parent']], weight_bounds=bound)
                    tree.insert(row_code_dict[code_info[code]['parent']], code, weight_bounds=bound)
                    
                elif row_code_dict[code_info[code]['parent']] in [node.name for node in tree.get_all_nodes()]:
                    tree.insert(row_code_dict[code_info[code]['parent']], code, weight_bounds=bound)
                
            else:
                if code not in [node.name for node in tree.get_all_nodes()]:
                    tree.insert('Universe', code, weight_bounds=bound)
                    tree.insert(code, code, weight_bounds=bound)
                elif code not in [node.name for node in tree.get_all_nodes()]:
                    tree.insert(code, code, weight_bounds=bound)
                    
        elif code_info[code]['level'] == 2:
            tree.insert(code, code, weight_bounds=bound)
            
    
    tree.draw()
    return tree

# 데이터 로드
df = pd.read_csv('로보리치 투자유니버스_수정.csv', encoding='cp949')

# 종목코드-종목설명 딕셔너리 생성
stock_dict = {}
for _, row in df.iterrows():
    code = str(row['종목 코드'])
    if len(code) < 6:
        code = '0' * (6 - len(code)) + code
    stock_dict[row['종목 설명']] = code

# 사용자 입력
codes = input('종목 코드를 입력하세요(","를 기준으로 적어주세요):').split(',')
risk_level = int(input('투자자의 위험도를 입력하세요:'))


# 트리 생성
investment_tree = build_investment_tree(codes, risk_level, df)


종목 코드를 입력하세요(","를 기준으로 적어주세요):69500,139260,161510,273130,439870,251340,114260
투자자의 위험도를 입력하세요:4
Creating Tree with root: Universe
Initializing Node: Universe with params: {}
Node Universe initialized with weight bounds: (0, 1)
Tree initialized with root node: Node(Universe, weight_bounds=(0, 1))
----------------
{'69500': {'level': 2.0, 'parent': 1.0, 'risk': 4}, '139260': {'level': 3.0, 'parent': 1.0, 'risk': 4}, '161510': {'level': 3.0, 'parent': 1.0, 'risk': 5}, '273130': {'level': 3.0, 'parent': 36.0, 'risk': 1}, '439870': {'level': 3.0, 'parent': 36.0, 'risk': 2}, '251340': {'level': 3.0, 'parent': 48.0, 'risk': 4}, '114260': {'level': 3.0, 'parent': 36.0, 'risk': 2}}
----------------
['139260', '161510', '273130', '439870', '251340', '114260', '69500']
Getting all nodes in the tree
Inserting child 69500 under parent Universe with params: {'weight_bounds': (0, 1)}
Initializing Node: 69500 with params: {'weight_bounds': (0, 1)}
Node 69500 initialized with weight bounds: (0, 1)
Addi